In [6]:
# ===============================
# SETUP — Importação de bibliotecas
# ===============================

from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns

from scipy.stats import chi2_contingency
from pathlib import Path

# Ajustes visuais
plt.style.use("default")
sns.set_theme()

print("Bibliotecas carregadas com sucesso!")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Bibliotecas carregadas com sucesso!


In [7]:
# ===============================
# Caminho da pasta onde seus CSVs estão
# ===============================

raw_path = "/content/drive/MyDrive/ESPA5_project/data/"

# Verificar arquivos existentes
print("Arquivos no diretório:")
print(os.listdir(raw_path))

# ===============================
# Carregando os arquivos CSV
# ===============================

collisions = pd.read_csv(
    raw_path + "Motor_Vehicle_Collisions_-_Crashes.csv",
    low_memory=False
)

ev = pd.read_csv(
    raw_path + "Electric_Vehicle_Population_Data.csv",
    low_memory=False
)

print("Dados carregados com sucesso!")
collisions.head()


Arquivos no diretório:
['Electric_Vehicle_Population_Data.csv', 'Motor_Vehicle_Collisions_-_Crashes.csv']
Dados carregados com sucesso!


,CRASH DATE,CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,...,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5
0,09/11/2021,2:39,NaN,NaN,NaN,NaN,NaN,WHITESTONE EXPRESSWAY,20 AVENUE,NaN,...,Unspecified,NaN,NaN,NaN,4455765,Sedan,Sedan,NaN,NaN,NaN
1,03/26/2022,11:45,NaN,NaN,NaN,NaN,NaN,QUEENSBORO BRIDGE UPPER,NaN,NaN,...,NaN,NaN,NaN,NaN,4513547,Sedan,NaN,NaN,NaN,NaN
2,11/01/2023,1:29,BROOKLYN,11230,40.62179,-73.970024,"(40.62179, -73.970024)",OCEAN PARKWAY,AVENUE K,NaN,...,Unspecified,Unspecified,NaN,NaN,4675373,Moped,Sedan,Sedan,NaN,NaN
3,06/29/2022,6:55,NaN,NaN,NaN,NaN,NaN,THROGS NECK BRIDGE,NaN,NaN,...,Unspecified,NaN,NaN,NaN,4541903,Sedan,Pick-up Truck,NaN,NaN,NaN
4,09/21/2022,13:21,NaN,NaN,NaN,NaN,NaN,BROOKLYN BRIDGE,NaN,NaN,...,Unspecified,NaN,NaN,NaN,4566131,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN,NaN


In [8]:
# ===============================
# LIMPEZA DE DADOS — Parte 1
# Padronização das colunas, datas, tipos, duplicatas e missing values
# ===============================

# 1) Função para padronizar nomes das colunas
def clean_columns(df):
    df = df.copy()
    df.columns = (
        df.columns
        .str.strip()
        .str.lower()
        .str.replace(" ", "_")
        .str.replace("[^0-9a-zA-Z_]", "", regex=True)
    )
    return df

# Aplicando ao dataset
collisions = clean_columns(collisions)
ev = clean_columns(ev)

print("Colunas padronizadas!")

# 2) Tratar datas (colunas variam dependendo da versão do dataset)
date_cols = ["crash_date", "crash_time"]

for col in date_cols:
    if col in collisions.columns:
        collisions[col] = pd.to_datetime(collisions[col], errors="coerce")

# Criar coluna 'hour' a partir de crash_time
if "crash_time" in collisions.columns:
    collisions["hour"] = pd.to_datetime(
        collisions["crash_time"].astype(str),
        errors="coerce"
    ).dt.hour

print("Datas e horários tratados!")

# 3) Remover duplicatas
before = collisions.shape[0]
collisions = collisions.drop_duplicates()
after = collisions.shape[0]

print(f"Removidas {before - after} linhas duplicadas.")

# 4) Relatório de valores ausentes
missing = collisions.isna().mean().sort_values(ascending=False)
print("\n==== MISSING VALUES (%) ====")
print((missing * 100).round(2))

# 5) Convertendo colunas numéricas (exemplo comum)
num_cols = [
    "number_of_persons_injured",
    "number_of_persons_killed",
    "number_of_pedestrians_injured",
    "number_of_cyclists_injured",
    "number_of_motorists_injured"
]

for col in num_cols:
    if col in collisions.columns:
        collisions[col] = pd.to_numeric(collisions[col], errors="coerce").fillna(0)

print("\nConversão de tipos numéricos concluída!")

collisions.head()


Colunas padronizadas!


/tmp/ipython-input-867464082.py:29: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  collisions[col] = pd.to_datetime(collisions[col], errors="coerce")


Datas e horários tratados!
Removidas 0 linhas duplicadas.

==== MISSING VALUES (%) ====
vehicle_type_code_5              99.56
contributing_factor_vehicle_5    99.55
vehicle_type_code_4              98.41
contributing_factor_vehicle_4    98.35
vehicle_type_code_3              93.05
contributing_factor_vehicle_3    92.76
off_street_name                  82.38
cross_street_name                38.21
zip_code                         30.65
borough                          30.64
on_street_name                   21.78
vehicle_type_code_2              20.06
contributing_factor_vehicle_2    16.05
latitude                         10.83
location                         10.83
longitude                        10.83
vehicle_type_code_1               0.73
contributing_factor_vehicle_1     0.35
number_of_persons_killed          0.00
number_of_persons_injured         0.00
crash_time                        0.00
crash_date                        0.00
number_of_pedestrians_killed      0.00
number_of_pedes

,crash_date,crash_time,borough,zip_code,latitude,longitude,location,on_street_name,cross_street_name,off_street_name,...,contributing_factor_vehicle_3,contributing_factor_vehicle_4,contributing_factor_vehicle_5,collision_id,vehicle_type_code_1,vehicle_type_code_2,vehicle_type_code_3,vehicle_type_code_4,vehicle_type_code_5,hour
0,2021-09-11,2025-11-14 02:39:00,NaN,NaN,NaN,NaN,NaN,WHITESTONE EXPRESSWAY,20 AVENUE,NaN,...,NaN,NaN,NaN,4455765,Sedan,Sedan,NaN,NaN,NaN,2
1,2022-03-26,2025-11-14 11:45:00,NaN,NaN,NaN,NaN,NaN,QUEENSBORO BRIDGE UPPER,NaN,NaN,...,NaN,NaN,NaN,4513547,Sedan,NaN,NaN,NaN,NaN,11
2,2023-11-01,2025-11-14 01:29:00,BROOKLYN,11230,40.62179,-73.970024,"(40.62179, -73.970024)",OCEAN PARKWAY,AVENUE K,NaN,...,Unspecified,NaN,NaN,4675373,Moped,Sedan,Sedan,NaN,NaN,1
3,2022-06-29,2025-11-14 06:55:00,NaN,NaN,NaN,NaN,NaN,THROGS NECK BRIDGE,NaN,NaN,...,NaN,NaN,NaN,4541903,Sedan,Pick-up Truck,NaN,NaN,NaN,6
4,2022-09-21,2025-11-14 13:21:00,NaN,NaN,NaN,NaN,NaN,BROOKLYN BRIDGE,NaN,NaN,...,NaN,NaN,NaN,4566131,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN,NaN,13
